In [3]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE, RandomOverSampler, ADASYN, BorderlineSMOTE, KMeansSMOTE, SVMSMOTE
from imblearn.under_sampling import ClusterCentroids, CondensedNearestNeighbour, EditedNearestNeighbours
from imblearn.under_sampling import RepeatedEditedNearestNeighbours, TomekLinks, AllKNN, InstanceHardnessThreshold
from imblearn.under_sampling import NearMiss, NeighbourhoodCleaningRule, OneSidedSelection, RandomUnderSampler

In [4]:
df_iot = pd.read_csv('C:/Users/Desktop/Documents/2024-1/aula 1/aula/RT_IOT2022', index_col =0)
df_iot.head

<bound method NDFrame.head of       id.orig_p  id.resp_p proto service  flow_duration  fwd_pkts_tot  \
0         38667       1883   tcp    mqtt      32.011598             9   
1         51143       1883   tcp    mqtt      31.883584             9   
2         44761       1883   tcp    mqtt      32.124053             9   
3         60893       1883   tcp    mqtt      31.961063             9   
4         51087       1883   tcp    mqtt      31.902362             9   
...         ...        ...   ...     ...            ...           ...   
2005      59247      63331   tcp       -       0.000006             1   
2006      59247      64623   tcp       -       0.000007             1   
2007      59247      64680   tcp       -       0.000006             1   
2008      59247      65000   tcp       -       0.000006             1   
2009      59247      65129   tcp       -       0.000006             1   

      bwd_pkts_tot  fwd_data_pkts_tot  bwd_data_pkts_tot  fwd_pkts_per_sec  \
0              

In [5]:
#Verificar o tamanho do arquivo
df_iot.shape

(123117, 84)

In [6]:
colunas_str = df_iot.select_dtypes(include=["object"]).columns
colunas_str

Index(['proto', 'service', 'Attack_type'], dtype='object')

In [7]:
#Remover colunas desejadas
df_iot.drop(columns=['proto', 'service'], inplace=True)
df_iot.head()

,id.orig_p,id.resp_p,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,...,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,Attack_type
0,38667,1883,32.011598,9,5,3,3,0.281148,0.156193,0.437341,...,0.0,2.972918e+07,2.972918e+07,2.972918e+07,2.972918e+07,0.0,64240,26847,502,MQTT_Publish
1,51143,1883,31.883584,9,5,3,3,0.282277,0.156821,0.439097,...,0.0,2.985528e+07,2.985528e+07,2.985528e+07,2.985528e+07,0.0,64240,26847,502,MQTT_Publish
2,44761,1883,32.124053,9,5,3,3,0.280164,0.155647,0.435811,...,0.0,2.984215e+07,2.984215e+07,2.984215e+07,2.984215e+07,0.0,64240,26847,502,MQTT_Publish
3,60893,1883,31.961063,9,5,3,3,0.281593,0.156440,0.438033,...,0.0,2.991377e+07,2.991377e+07,2.991377e+07,2.991377e+07,0.0,64240,26847,502,MQTT_Publish
4,51087,1883,31.902362,9,5,3,3,0.282111,0.156728,0.438839,...,0.0,2.981470e+07,2.981470e+07,2.981470e+07,2.981470e+07,0.0,64240,26847,502,MQTT_Publish


In [12]:
#Separar 50% dos dados para tratamento e teste
df_amostra = df_iot.sample(frac=0.5, random_state=42)
df_amostra.shape

(61558, 82)

In [13]:
#Para me mostrar somente os 15 primeiros elementos
df_amostra.head(15)

,id.orig_p,id.resp_p,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,...,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,Attack_type
55444,7315,21,0.000001,1,1,1,0,8.388608e+05,8.388608e+05,1.677722e+06,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,64,0,64,DOS_SYN_Hping
15766,18609,21,0.000004,1,1,1,0,2.621440e+05,2.621440e+05,5.242880e+05,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,64,0,64,DOS_SYN_Hping
63311,14663,21,0.000002,1,1,1,0,4.660338e+05,4.660338e+05,9.320676e+05,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,64,0,64,DOS_SYN_Hping
3245,37827,1883,62.052435,9,5,3,3,1.450390e-01,8.057700e-02,2.256160e-01,...,0.0,5.981909e+07,5.981909e+07,5.981909e+07,5.981909e+07,0.0,64240,26847,502,MQTT_Publish
94007,22348,21,0.000000,1,0,1,0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,64,0,64,DOS_SYN_Hping
78072,37150,21,0.000005,1,1,1,0,1.997288e+05,1.997288e+05,3.994575e+05,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,64,0,64,DOS_SYN_Hping
78191,36115,21,0.000005,1,1,1,0,1.997288e+05,1.997288e+05,3.994575e+05,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,64,0,64,DOS_SYN_Hping
39338,42253,21,0.000004,1,1,1,0,2.467238e+05,2.467238e+05,4.934475e+05,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,64,0,64,DOS_SYN_Hping
721,36242,1334,0.000009,1,1,0,0,1.103764e+05,1.103764e+05,2.207528e+05,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,1024,0,1024,NMAP_XMAS_TREE_SCAN
2554,49275,53,0.021367,2,2,2,2,9.360196e+01,9.360196e+01,1.872039e+02,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0,0,0,Thing_Speak


In [11]:
# Começando a fazer os testes e treinamentos 
indices = ~df_iot.index.isin(df_amostra.index)
df_teste = df_iot[indices]
df_teste.head(15)

,id.orig_p,id.resp_p,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,...,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,Attack_type
566,56425,1883,61.135028,9,5,3,3,0.147215,0.081786,0.229001,...,0.0,5.872618e+07,5.872618e+07,5.872618e+07,5.872618e+07,0.0,64240,26847,502,MQTT_Publish
651,42353,1883,28.060861,9,5,3,3,0.320731,0.178184,0.498916,...,0.0,2.575971e+07,2.575971e+07,2.575971e+07,2.575971e+07,0.0,64240,26847,502,MQTT_Publish
915,54085,1883,62.127916,10,6,3,4,0.160958,0.096575,0.257533,...,0.0,5.975666e+07,5.975666e+07,5.975666e+07,5.975666e+07,0.0,64240,26847,502,MQTT_Publish
1235,41369,1883,61.982733,10,6,3,4,0.161335,0.096801,0.258136,...,0.0,5.990369e+07,5.990369e+07,5.990369e+07,5.990369e+07,0.0,64240,26847,502,MQTT_Publish
1504,44743,1883,13.953928,9,5,3,3,0.644980,0.358322,1.003302,...,0.0,1.185618e+07,1.185618e+07,1.185618e+07,1.185618e+07,0.0,64240,26847,502,MQTT_Publish
1782,58115,1883,20.959591,10,6,3,4,0.477109,0.286265,0.763374,...,0.0,1.886782e+07,1.886782e+07,1.886782e+07,1.886782e+07,0.0,64240,26847,502,MQTT_Publish
1801,52997,1883,47.957955,10,6,3,4,0.208516,0.125110,0.333626,...,0.0,4.588572e+07,4.588572e+07,4.588572e+07,4.588572e+07,0.0,64240,26847,502,MQTT_Publish
2036,37363,1883,23.028695,9,5,3,3,0.390817,0.217120,0.607937,...,0.0,2.074595e+07,2.074595e+07,2.074595e+07,2.074595e+07,0.0,64240,26847,502,MQTT_Publish
2041,42253,1883,60.997583,10,6,3,4,0.163941,0.098365,0.262305,...,0.0,5.891470e+07,5.891470e+07,5.891470e+07,5.891470e+07,0.0,64240,26847,502,MQTT_Publish
2054,53451,1883,23.902184,10,6,3,4,0.418372,0.251023,0.669395,...,0.0,2.187053e+07,2.187053e+07,2.187053e+07,2.187053e+07,0.0,64240,26847,502,MQTT_Publish


In [14]:
#Separar dados de predição 
#abaixo os preditores
x = df_amostra.drop(columns={'Attack_type'})
#O drop se utiliza para remover colunas.
y = df_amostra['Attack_type']

In [15]:
# Verifivar quais os valores e quantidades de elementos da coluna anterior 
y.value_counts()

Attack_type
DOS_SYN_Hping                 47425
Thing_Speak                    3982
ARP_poisioning                 3807
MQTT_Publish                   2073
NMAP_UDP_SCAN                  1287
NMAP_OS_DETECTION              1010
NMAP_XMAS_TREE_SCAN             995
NMAP_TCP_scan                   542
DDOS_Slowloris                  268
Wipro_bulb                      140
Metasploit_Brute_Force_SSH       15
NMAP_FIN_SCAN                    14
Name: count, dtype: int64

In [16]:
df_teste['Attack_type'].value_counts()

Attack_type
DOS_SYN_Hping          44136
Thing_Speak              673
ARP_poisioning           595
MQTT_Publish             138
NMAP_UDP_SCAN             27
NMAP_OS_DETECTION          7
NMAP_XMAS_TREE_SCAN        7
NMAP_TCP_scan              3
Name: count, dtype: int64

In [17]:
#Data frame mais organizado e bem feito
pd.DataFrame({"Nome": y.value_counts().index,
              "Qte." : y.value_counts().values,
              "% Total": round(y.value_counts(normalize=True) * 100, 2).values})

,Nome,Qte.,% Total
0,DOS_SYN_Hping,47425,77.04
1,Thing_Speak,3982,6.47
2,ARP_poisioning,3807,6.18
3,MQTT_Publish,2073,3.37
4,NMAP_UDP_SCAN,1287,2.09
5,NMAP_OS_DETECTION,1010,1.64
6,NMAP_XMAS_TREE_SCAN,995,1.62
7,NMAP_TCP_scan,542,0.88
8,DDOS_Slowloris,268,0.44
9,Wipro_bulb,140,0.23


In [20]:
### Over Sampling ###

smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample( x , y)
X_smote.shape




(569100, 81)